<strong><b><font size="6">Optimization with Linear Programming</font></b></strong>


In [1]:
import pulp
from pulp import *

# Resource Planing Example

__Problem 1:__ A bakery makes cakes and pies every day. It can make a total of 30 items in one day, which at least must be 5 cakes and 10 pies for its planned customers. The profit on each cake is 1.5 and the profit on each pie is 2.00. How many should be made to maximize the profit?

In [3]:
# Initialize the model
model = LpProblem("Maximize Bakery Profits", sense= LpMaximize)

# Define Decision Variables, 
C = LpVariable('C', lowBound=5, upBound=None, cat='Integer')
P = LpVariable('P', lowBound=10, upBound=None, cat='Integer')

# Define Objective Function
model += 1.5 * C + 2 * P 

# Define Constraints

model += C + P <= 30

# Solve Model
model.solve()
print(LpStatus[model.status])
print("Produce {} Cake ".format(C.varValue))
print("Produce {} Pie ".format(P.varValue))
print(value(model.objective))

Optimal
Produce 5.0 Cake 
Produce 25.0 Pie 
57.5


__Problem 2:__ A bakery makes cakes and pies every day of a month. There is: 1 oven, 2 bakers, 1 packaging packer that works only 22 days of the month. The cake requires to use the oven for 1 day and the pie requires 0.5 day. Each baker needs to work for cake 0.5 days and pie 2 days. Packer needs to work for cake 1 day and pie 0.5 days. The profit on each cake is 15 and the profit on each pie is 12. How many should be made to maximize the profit under given conditions?

In [4]:
# Initialize the model
model = LpProblem("Maximize Bakery Profits", sense= LpMaximize)

# Define Decision Variables, 
Cake = LpVariable('Cake', lowBound=0, upBound=None, cat='Integer')
Pie = LpVariable('Pie', lowBound=0, upBound=None, cat='Integer')

# Define Objective Function
model += 15 * Cake + 12 * Pie

# Define Constraints

model += 1 * Cake + 0.5 * Pie <= 30
model += 0.5 * Cake + 2 * Pie <= 60
model += 1 * Cake + 0.5 * Pie <= 22

# Solve Model
model.solve()
print(LpStatus[model.status])
print("Produce {} Cake ".format(Cake.varValue))
print("Produce {} Pie ".format(Pie.varValue))
print(value(model.objective))

Optimal
Produce 8.0 Cake 
Produce 28.0 Pie 
456.0


# Scheduling Example# 

A post office is looking to hire postman, with the requirements to work 5 consecutive days and then 2 days off. The objective is to hire the minimum number of workers and the estimated number of postmen needed for each day is Monday: 25, Tuesday: 32, Wednesday: 22, Thursday: 18, Friday: 24, Saturday: 12, Sunday: 14. What would be the minimum number of postmen to hire?

In [5]:
#Initialize model
model = LpProblem("Minimize Number of Workers", LpMinimize)

#Define decision variables
days = list(range(7))
x = LpVariable.dicts('workers_', days, lowBound=0, cat='Integer')

#Define model
model += lpSum([x[i] for i in days])

# Define constraints
model += x[0] + x[3] + x[4] + x[5] + x[6] >= 25
model += x[0] + x[1] + x[4] + x[5] + x[6] >= 32
model += x[0] + x[1] + x[2] + x[5] + x[6] >= 22
model += x[0] + x[1] + x[2] + x[3] + x[6] >= 18
model += x[0] + x[1] + x[2] + x[3] + x[4] >= 24
model += x[1] + x[2] + x[3] + x[4] + x[5] >= 12
model += x[2] + x[3] + x[4] + x[5] + x[6] >= 14

# Solve model
model.solve()

#Print model status
print("Status:", LpStatus[model.status])

#Print solution variables
for variable in model.variables():
    print ("{} = {}".format(variable.name, variable.varValue))

Status: Optimal
workers__0 = 7.0
workers__1 = 7.0
workers__2 = 0.0
workers__3 = 0.0
workers__4 = 10.0
workers__5 = 0.0
workers__6 = 8.0


# Location Problem Example

Assume that you need to optimize a manufacturing company's supply chain network across 5 selling locations to meet with demand by location at the lowest cost. You can decide the plant size in each location where the options are low capacity and high capacity. One possibility is to set up a facility in each region with an advantage of low transportation costs and a disadvantage of having production plans sized to meet local demands and not exploiting the economies of scale. Another possibility is to set up a few manufacturing plants with an advantage of economies of scale but requiring higher transportation costs. Given, you have the estimated demand for each location, variable costs of transportation from one plant to another, fixed costs of having a plant-based on its size and the production capacity based on plant size is 500 for low capacity and 1500 for high capacity. How would you solve this problem with the minimum cost?

In [6]:
import pandas as pd

In [7]:
#Given:
dmd = {'Demand': {'A': 145.4,
  'B': 84.1,
  'C': 156.4,
  'D': 1676.8,
  'E': 2719.6}}

var_co = {'A': {'A': 8, 'B': 14, 'C': 23, 'D': 21, 'E': 12},
 'B': {'A': 14, 'B': 6, 'C': 13, 'D': 14, 'E': 13},
 'C': {'A': 21, 'B': 13, 'C': 8, 'D': 9, 'E': 17},
 'D': {'A': 21, 'B': 14, 'C': 10, 'D': 3, 'E': 20},
 'E': {'A': 12, 'B': 13, 'C': 22, 'D': 20, 'E': 6}}

fix_co = {'High_C': {'A': 4730,
  'B': 7270,
  'C': 3080,
  'D': 9100,
  'E': 9500},
 'Low_C': {'A': 3230,
  'B': 4980,
  'C': 2110,
  'D': 6230,
  'E': 6500}}

cap = {'High_C': {'A': 1500,
  'B': 1500,
  'C': 1500,
  'D': 1500,
  'E': 1500},
 'Low_C': {'A': 500,
  'B': 500,
  'C': 500,
  'D': 500,
  'E': 500}}

demand = pd.DataFrame(dmd)
var_cost = pd.DataFrame(var_co)
fix_cost = pd.DataFrame(fix_co)
capacity = pd.DataFrame(cap)

In [8]:
# Initialize model
model = LpProblem("Facility Location Problem", LpMinimize)

#Define decision variables
loc = ['A', 'B', 'C', 'D', 'E']
size = ['Low_C','High_C']

x = LpVariable.dicts('production_', [(i,j) for i in loc for j in loc], lowBound=0, upBound=None, cat='Continuous')
y = LpVariable.dicts('plant_', [(i,s) for s in size for i in loc], lowBound=None, upBound=None, cat='Binary')
#Define model
model += (lpSum([fix_cost.loc[i,s] * y[(i,s)] for s in size for i in loc])
          + lpSum([var_cost.loc[i,j] * x[(i,j)] for i in loc for j in loc]))

# Define constraints
for j in loc:
    model += lpSum([x[(i, j)] for i in loc]) == demand.loc[j,'Demand']
for i in loc:
    model += lpSum([x[i, j] for j in loc]) <= lpSum([capacity.loc[i,s] * y[i,s] for s in size])
    
# Solve
model.solve()

# Print results for production quantities
sol = [{'prod':'{} to {}'.format(i,j), 'quantity':x[(i,j)].varValue} 
     for i in loc for j in loc]
print(pd.DataFrame(sol))

# Print results for plant capacities based on location
sol = [{'lowCap':y[(i,size[0])].varValue, 'highCap':y[(i,size[1])].varValue}
     for i in loc]
print(pd.DataFrame(sol, index=loc))

# Print the Objective Value
print('Objective = ', value(model.objective))

# Print Sensitivity Analysis Results: Dual Value and Slack
print('Sensitivity Analysis')
sol = [{'name':name, 'dual value':c.pi, 'slack': c.slack}
for name, c in model.constraints.items()]
print(pd.DataFrame(sol))

      prod  quantity
0   A to A     145.4
1   A to B       0.0
2   A to C       0.0
3   A to D       0.0
4   A to E    1219.6
5   B to A       0.0
6   B to B       0.0
7   B to C       0.0
8   B to D       0.0
9   B to E       0.0
10  C to A       0.0
11  C to B      84.1
12  C to C     156.4
13  C to D     176.8
14  C to E       0.0
15  D to A       0.0
16  D to B       0.0
17  D to C       0.0
18  D to D    1500.0
19  D to E       0.0
20  E to A       0.0
21  E to B       0.0
22  E to C       0.0
23  E to D       0.0
24  E to E    1500.0
   lowCap  highCap
A     0.0      1.0
B     0.0      0.0
C     1.0      0.0
D     0.0      1.0
E     0.0      1.0
Objective =  58850.9
Sensitivity Analysis
   name  dual value  slack
0   _C1         8.0   -0.0
1   _C2        13.0   -0.0
2   _C3         8.0   -0.0
3   _C4        10.0   -0.0
4   _C5        12.0   -0.0
5   _C6         0.0  135.0
6   _C7        -7.0   -0.0
7   _C8         0.0   82.7
8   _C9        -7.0   -0.0
9  _C10        -6.0   -0.0
